<a href="https://colab.research.google.com/github/ameek2/CISC662/blob/master/tensorflowBenchmarkRun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
g = tf.Graph()
with g.as_default():
  mnist = tf.keras.datasets.mnist

  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train, x_test = x_train/ 255.0, x_test/255.0
  model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
  ])

  predictions = model(x_train[:1]).numpy()

  tf.nn.softmax(predictions).numpy()
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  loss_fn(y_train[:1], predictions).numpy()

  model.compile(optimizer='adam',loss=loss_fn, metrics= ['accuracy'])
  model.fit(x_train,y_train,epochs=5)
  model.evaluate(x_test, y_test, verbose=2)
  probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
  probability_model(x_test[:5])

for op in g.get_operations():
  flops = ops.get_stats_for_node_def(g, op.node_def, 'flops').value
  if flops is not None:
    print ('TF stats gives', flops)
  
  tf.config.experimental.get_memory_info(g)

11490434/11490434 [==============================] - 0s 0us/step


AttributeError: ignored

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.python.framework.ops as ops
import os
import tensorflow_datasets as tfds

!pip show keras #2.2.4 and upwards automatically uses multiple cores. without using multiple cores roofline models are useless so we need to check this
!pip install keras-flops


Name: keras
Version: 2.9.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.8/dist-packages
Requires: 
Required-by: tensorflow, keras-vis
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Imports. Will cull as needed

Basic code stolen from
https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb#scrollTo=3wF5wszaj97Y

resnet18

resnet34

resnet50

resnet101

resnet152

vgg11

vgg13

vgg16

vgg19

wide_resnet50_2

alexnet

mobilenet_v2

efficientnet_b0 

above is the list of models tested in pytorch. below is the intersecting models available built-in from keras.applications: https://keras.io/api/applications/

code uses the library keras-flops from tokusumi on github: https://github.com/tokusumi/keras-flops

In [ ]:
#USE THIS CELL WITH GPUs

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from keras_flops import get_flops

# build model
inp = Input((32, 32, 3))
x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inp)
x = Conv2D(64, (3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
out = Dense(10, activation="softmax")(x)
model = Model(inp, out)

from keras.applications import ResNet50
from keras.applications import ResNet50V2
from keras.applications import ResNet101
from keras.applications import ResNet101V2
from keras.applications import ResNet152
from keras.applications import ResNet152V2
from keras.applications import Xception
from keras.applications import InceptionV3
from keras.applications import InceptionResNetV2
from keras.applications import DenseNet121
from keras.applications import DenseNet169
from keras.applications import DenseNet201
from keras.applications import NASNetMobile
from keras.applications import NASNetLarge
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import MobileNet
from keras.applications import MobileNetV2
from keras.applications import EfficientNetB0
from keras.applications import EfficientNetB1
from keras.applications import EfficientNetB2
from keras.applications import EfficientNetB3
from keras.applications import EfficientNetB4
from keras.applications import EfficientNetB5
from keras.applications import EfficientNetB6
from keras.applications import EfficientNetB7
from keras.applications import EfficientNetV2B0
from keras.applications import EfficientNetV2B1
from keras.applications import EfficientNetV2B2
from keras.applications import EfficientNetV2B3
from keras.applications import EfficientNetV2S
from keras.applications import EfficientNetV2M
from keras.applications import EfficientNetV2L

'''
from keras.applications import ConvNeXtTiny
from keras.applications import ConvNeXtSmall
from keras.applications import ConvNeXtBase
from keras.applications import ConvNeXtLarge
from keras.applications import ConvNeXtXLarge
'''

model_list = [
  "ResNet50",
  "ResNet50V2",
  "ResNet101",
  "ResNet101V2",
  "ResNet152",
  "ResNet152V2",
  "Xception",
  "InceptionV3",
  "InceptionResNetV2",
  "DenseNet121",
  "DenseNet169",
  "DenseNet201",
  "NASNetMobile",
  "NASNetLarge",
  "VGG16",
  "VGG19",
  "MobileNet",
  "MobileNetV2",
  "EfficientNetB0",
  "EfficientNetB1",
  "EfficientNetB2",
  "EfficientNetB3",
  "EfficientNetB4",
  "EfficientNetB5",
  "EfficientNetB6",
  "EfficientNetB7",
  "EfficientNetV2B0",
  "EfficientNetV2B1",
  "EfficientNetV2B2",
  "EfficientNetV2B3",
  "EfficientNetV2S",
  "EfficientNetV2M",
  "EfficientNetV2L"
]

for i in model_list:
  to_be_exec = """
model = """ + i + """(weights="imagenet")
print('""" + i + """, flops: ', get_flops(model, batch_size=1))
print('""" + i + """, Memory used: ', tf.config.experimental.get_memory_info('GPU:0'))
tf.config.experimental.reset_memory_stats('GPU:0')
"""

  exec(to_be_exec)




# Calculate FLOPS
flops = get_flops(model, batch_size=1)
#print(f"FLOPS: {flops / 10 ** 9:.03} G")
#print(flops)
# >>> FLOPS: 0.0338 G

# gpus = tf.config.experimental.list_physical_devices('GPU')
# print (gpus)
# tpus = tf.config.experimental.list_physical_devices('TPU')
# print (tpus)
# cpus = tf.config.experimental.list_physical_devices('CPU')
# print (cpus)
# try:
#  device_name = os.environ['COLAB_TPU_ADDR']
#  TPU_ADDRESS = 'grpc://' + device_name
#  print('Found TPU at: {}'.format(TPU_ADDRESS))
# except KeyError:
#   device_name = tf.test.gpu_device_name()
#   print (device_name)
#   if device_name != '/device:GPU:0':
#     print ('NOT GPU')
#   print('TPU not found')

# with tf.device('/GPU:0'):
#     tensorflow_dataset = tf.constant(numpy_dataset)

'''
model = ResNet50(weights="imagenet")
print("Resnet50, flops: ", get_flops(model, batch_size=1))
print("Resnet50, Memory used: ", tf.config.experimental.get_memory_info('GPU:0'))
tf.config.experimental.reset_memory_stats('GPU:0')

model = VGG16(weights="imagenet", include_top=False)
print("VGG16, flops: ", get_flops(model, batch_size=1))
print("VGG16, Memory used: ", tf.config.experimental.get_memory_info('GPU:0'))
tf.config.experimental.reset_memory_stats('GPU:0')

model = VGG19(weights="imagenet")
print("VGG19, flops: ", get_flops(model, batch_size=1))
print("VGG19, Memory used: ", tf.config.experimental.get_memory_info('GPU:0'))
tf.config.experimental.reset_memory_stats('GPU:0')

model = MobileNetV2()
print("MobileNetV2, flops: ", get_flops(model, batch_size=1))
print("MobileNetV2, Memory used: ", tf.config.experimental.get_memory_info('GPU:0'))
tf.config.experimental.reset_memory_stats('GPU:0')

model = EfficientNetB0()
print("EfficientNetB0, flops: ", get_flops(model, batch_size=1))
print("EfficientNetB0, Memory used: ", tf.config.experimental.get_memory_info('GPU:0'))
'''

ResNet50, flops:  7755299312
ResNet50, Memory used:  {'current': 2859374592, 'peak': 2867564544}
ResNet50V2, flops:  6992475632
ResNet50V2, Memory used:  {'current': 2040826880, 'peak': 2985594880}
ResNet101, flops:  15198664176
ResNet101, Memory used:  {'current': 2252812544, 'peak': 2261004544}
ResNet101V2, flops:  14434134512
ResNet101V2, Memory used:  {'current': 2465134592, 'peak': 2473326592}
ResNet152, flops:  22646426096
ResNet152, Memory used:  {'current': 2208280320, 'peak': 2757563136}
ResNet152V2, flops:  21879789040
ResNet152V2, Memory used:  {'current': 2494334720, 'peak': 2510915328}


In [5]:
#USE THIS CELL WITH TPUs

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from keras_flops import get_flops

# build model  - this is unnecessary, can delete
inp = Input((32, 32, 3))
x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inp)
x = Conv2D(64, (3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
out = Dense(10, activation="softmax")(x)
model = Model(inp, out)

from keras.applications import ResNet50
from keras.applications import ResNet50V2
from keras.applications import ResNet101
from keras.applications import ResNet101V2
from keras.applications import ResNet152
from keras.applications import ResNet152V2
from keras.applications import Xception
from keras.applications import InceptionV3
from keras.applications import InceptionResNetV2
from keras.applications import DenseNet121
from keras.applications import DenseNet169
from keras.applications import DenseNet201
from keras.applications import NASNetMobile
from keras.applications import NASNetLarge
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import MobileNet
from keras.applications import MobileNetV2
from keras.applications import EfficientNetB0
from keras.applications import EfficientNetB1
from keras.applications import EfficientNetB2
from keras.applications import EfficientNetB3
from keras.applications import EfficientNetB4
from keras.applications import EfficientNetB5
from keras.applications import EfficientNetB6
from keras.applications import EfficientNetB7
from keras.applications import EfficientNetV2B0
from keras.applications import EfficientNetV2B1
from keras.applications import EfficientNetV2B2
from keras.applications import EfficientNetV2B3
from keras.applications import EfficientNetV2S
from keras.applications import EfficientNetV2M
from keras.applications import EfficientNetV2L

'''
from keras.applications import ConvNeXtTiny
from keras.applications import ConvNeXtSmall
from keras.applications import ConvNeXtBase
from keras.applications import ConvNeXtLarge
from keras.applications import ConvNeXtXLarge
'''

model_list = [
  "ResNet50",
  "ResNet50V2",
  "ResNet101",
  "ResNet101V2",
  "ResNet152",
  "ResNet152V2",
  "Xception",
  "InceptionV3",
  "InceptionResNetV2",
  "DenseNet121",
  "DenseNet169",
  "DenseNet201",
  "NASNetMobile",
  "NASNetLarge",
  "VGG16",
  "VGG19",
  "MobileNet",
  "MobileNetV2",
  "EfficientNetB0",
  "EfficientNetB1",
  "EfficientNetB2",
  "EfficientNetB3",
  "EfficientNetB4",
  "EfficientNetB5",
  "EfficientNetB6",
  "EfficientNetB7",
  "EfficientNetV2B0",
  "EfficientNetV2B1",
  "EfficientNetV2B2",
  "EfficientNetV2B3",
  "EfficientNetV2S",
  "EfficientNetV2M",
  "EfficientNetV2L"
]

'''
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))
except KeyError:
  device_name = tf.test.gpu_device_name()
  print (device_name)
  if device_name != '/device:GPU:0':
    print ('NOT GPU')
  print('TPU not found')
'''

for i in model_list:
  to_be_exec = """
model = """ + i + """(weights="imagenet")
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
  model,
  strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)))
)
print('""" + i + """, flops: ', get_flops(tpu_model, batch_size=1))
print('""" + i + """, Memory used: ', tf.config.experimental.get_memory_info('GPU:0'))
tf.config.experimental.reset_memory_stats('GPU:0') 
"""
  #in the tf.config line above, change 'GPU:0' to whatever we figure out the TPU is named. Then this should be good to go.
  exec(to_be_exec)

# Calculate FLOPS
flops = get_flops(model, batch_size=1)
#print(f"FLOPS: {flops / 10 ** 9:.03} G")
#print(flops)
# >>> FLOPS: 0.0338 G

'''
David - you can do multiline comments by surrounding lines with triplets of double or single apostrophes
gpus = tf.config.experimental.list_physical_devices('GPU')
print (gpus)
tpus = tf.config.experimental.list_physical_devices('TPU')
print (tpus)
cpus = tf.config.experimental.list_physical_devices('CPU')
print (cpus)
'''

# with tf.device('/GPU:0'):
#     tensorflow_dataset = tf.constant(numpy_dataset)

'''
model = ResNet50(weights="imagenet")
print("Resnet50, flops: ", get_flops(model, batch_size=1))
print("Resnet50, Memory used: ", tf.config.experimental.get_memory_info('GPU:0'))
tf.config.experimental.reset_memory_stats('GPU:0')

model = VGG16(weights="imagenet", include_top=False)
print("VGG16, flops: ", get_flops(model, batch_size=1))
print("VGG16, Memory used: ", tf.config.experimental.get_memory_info('GPU:0'))
tf.config.experimental.reset_memory_stats('GPU:0')

model = VGG19(weights="imagenet")
print("VGG19, flops: ", get_flops(model, batch_size=1))
print("VGG19, Memory used: ", tf.config.experimental.get_memory_info('GPU:0'))
tf.config.experimental.reset_memory_stats('GPU:0')

model = MobileNetV2()
print("MobileNetV2, flops: ", get_flops(model, batch_size=1))
print("MobileNetV2, Memory used: ", tf.config.experimental.get_memory_info('GPU:0'))
tf.config.experimental.reset_memory_stats('GPU:0')

model = EfficientNetB0()
print("EfficientNetB0, flops: ", get_flops(model, batch_size=1))
print("EfficientNetB0, Memory used: ", tf.config.experimental.get_memory_info('GPU:0'))
'''

Resnet50, flops:  7755299312


ValueError: ignored